# 导入库

In [59]:
%matplotlib inline
import matplotlib  
matplotlib.use('Agg')
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (11, 11)  #set default figure size
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error,r2_score
import matplotlib.pylab as plt


C:\Users\FFF123\AppData\Local\Temp\ipykernel_9264\107850124.py:3: MatplotlibDeprecationWarning: Auto-close()ing of figures upon backend switching is deprecated since 3.8 and will be removed in 3.10.  To suppress this warning, explicitly call plt.close('all') first.
  matplotlib.use('Agg')


# 逆预测值（重采样）

In [60]:
# inverse predictive values
def inverse_scaler(y_scaled, y_true):   
    scaler =MinMaxScaler()              
    scaler.fit(np.reshape(yy.values,(len(yy),1)))      
    y_inverse = scaler.inverse_transform(y_scaled.reshape(-1,1))       
    return y_inverse       

# 导入数据

In [61]:
df = pd.read_csv('MP_data_0323_TN.csv', index_col=None, engine='c') 
print('original datasize:', df.shape)
df.dropna(inplace=True)
print('adjusted datasize:', df.shape)
print('data:')
df

original datasize: (222, 17)
adjusted datasize: (222, 17)
data:


,Time,Deep,Length,HRT,COD,TN,NH4-N,C/N,c(A),CSC,XLogP3,WI,RA(P),RA(F),RA(B),RA(A),RR(TN)
0,90,12.5,45.0,72.0,299.46,26.80,24.07,11.17,100.00,11,-0.1,885,48.0057,1.8966,9.9499,2.3363,0.6451
1,90,45.0,12.5,72.0,299.46,26.80,24.07,11.17,100.00,11,-0.1,885,62.3032,1.6632,6.7403,0.9921,0.6451
2,84,10.0,30.0,72.0,66.60,3.60,2.35,18.50,0.18,11,0.2,943,50.8357,0.9988,10.7969,7.0869,0.8028
3,84,10.0,90.0,72.0,66.60,3.60,2.35,18.50,0.18,11,0.2,943,46.2696,1.4269,11.2249,2.6160,0.8028
4,84,50.0,30.0,72.0,66.60,3.60,2.35,18.50,0.18,11,0.2,943,38.7070,0.6659,6.9918,1.5220,0.8028
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217,50,20.0,40.0,72.0,60.06,35.83,4.86,1.68,0.00,0,0.0,0,17.1306,6.3166,6.6000,4.9164,0.7311
218,210,40.0,40.0,48.0,570.32,32.08,20.14,17.78,0.00,0,0.0,0,57.2920,3.7780,10.2600,4.3550,0.4819
219,210,40.0,40.0,48.0,570.32,32.08,20.14,17.78,0.00,0,0.0,0,51.0730,2.2040,4.0930,2.2040,0.5313
220,210,40.0,40.0,48.0,570.32,32.08,20.14,17.78,0.00,0,0.0,0,58.2630,11.1570,5.7410,7.0060,0.3772


# 标准化数据，分割训练集，测试集，验证集

In [62]:
xx= df.iloc[:,0:-1]  # 特征 
yy=df.iloc[:,-1]   # 目标变量

zscore = StandardScaler()   #用于数据标准化处理的缩放器
scaler =MinMaxScaler()   #用于数据最小最大缩放处理的缩放器
xx_scaled = zscore.fit_transform(xx)  
yy_scaled = scaler.fit_transform(np.reshape(yy.values,(len(yy),1))) 

def split(xx_scaled, yy_scaled):      #定义一个函数用来划分训练集，测试集，验证集
     
    x_train, x_test_val, y_train, y_test_val = train_test_split(xx_scaled, yy_scaled, shuffle=True,test_size=0.3, random_state=1)
    #x_训练集，x_测试验证集，y_训练集，y_测试验证集                    划分前将数据集进行随机打乱，并按照6：4划分，且划分方式统一
    x_test, x_val, y_test, y_val = train_test_split(x_test_val, y_test_val, shuffle=True,test_size=0.25, random_state=1)
    #x_测试集，x_验证集，y_测试集，y_验证集                           划分前将数据集进行随机打乱，并按照3：1划分，且划分方式统一
    
    y2 = yy_scaled.ravel()     #将yy数组转化为一维数组并存在y2中
    ind_train = [np.argwhere(y2 == y_train[i])[0].item() for i in range(len(y_train)) if y_train[i] in y2]
                                                        #这是一个循环，遍历y_train中的每一个元素，判断y_train中的当前标签y_train[i]是否存在于y2中
                #找到y2中所有等于y_train[i]的元素的索引，返回一个二维数组，然后用[0].item()取得第一行赋值给ind_train生成列表
    ind_test = [np.argwhere(y2 == y_test[i])[0].item() for i in range(len(y_test)) if y_test[i] in y2]
    ind_val = [np.argwhere(y2 == y_val[i])[0].item() for i in range(len(y_val)) if y_val[i] in y2]
   
    y_train = y_train.ravel()       #将y_train数组转化为一维数组
    y_test = y_test.ravel()         #将y_text数组转化为一维数组
    y_val = y_val.ravel()           #将y_val数组转化为一维数组
    
    return x_train, x_test, x_val, y_train, y_test, y_val, xx, yy, np.asarray(ind_train), np.asarray(ind_test), np.asarray(ind_val)
    #返回训练集，测试集，验证集的特征数据，和标签（目标变量）数据，原特数据，原目标变量数据，以及转化为数组的ind_train，ind_train，ind_val

In [63]:
x_train, x_test, x_val, y_train, y_test,y_val, x_scaled, y_scaled, ind_train, ind_test, ind_val = split(xx_scaled, yy_scaled)

print('目标变量 is ', yy.name)
print('特征 are ', xx.columns.tolist())
print('样本数量 is ', xx.shape[0], '; 预测特征变量 is ', xx.shape[1])
print('训练集大小 is ', x_train.shape[0], '; 测试集大小 is ', x_test.shape[0],'; 验证集大小 is ', x_val.shape[0])

目标变量 is  RR(TN)
特征 are  ['Time', 'Deep', 'Length', 'HRT', 'COD', 'TN', 'NH4-N', 'C/N', 'c(A)', 'CSC', 'XLogP3', 'WI', 'RA(P)', 'RA(F)', 'RA(B)', 'RA(A)']
样本数量 is  222 ; 预测特征变量 is  16
训练集大小 is  155 ; 测试集大小 is  50 ; 验证集大小 is  17


# XGB

In [64]:
import pickle
# load xgboost model
path='Models/'
model_name = 'xgb.pkl'
with open(path+model_name,'rb') as f:
    xgb = pickle.load(f)
    
y_pred_train = xgb.predict(x_train)
y_pred_test = xgb.predict(x_test)    
y_pred_val = xgb.predict(x_val) 


# GPR

In [65]:
import pickle
# load gpr model
path='Models/'
model_name = 'gpr.pkl'
with open(path+model_name,'rb') as f:
    gpr = pickle.load(f)
    
y_pred_train = gpr.predict(x_train)    
y_pred_test = gpr.predict(x_test)    
y_pred_val = gpr.predict(x_val) 

FileNotFoundError: [Errno 2] No such file or directory: 'Models/gpr.pkl'

# Ga-Svm

In [ ]:
import pickle
# load gpr model
model_name = 'GA_SVM.pkl'
with open(path+model_name,'rb') as f:
    ga_svm = pickle.load(f)
    
y_pred_train = ga_svm.predict(x_train)    
y_pred_test = ga_svm.predict(x_test)    
y_pred_val = ga_svm.predict(x_val)  

# RF

In [ ]:
import pickle
# load rf model
path='Models/'
model_name = 'rf.pkl'
with open(path+model_name,'rb') as f:
    rf = pickle.load(f)
    
y_pred_train = rf.predict(x_train)    
y_pred_test = rf.predict(x_test)    
y_pred_val = rf.predict(x_val)  

# 敏感性分析

In [ ]:
def MSE(y_test, y_pred_test):
    
    return mean_squared_error(y_test, y_pred_test)

mse_value = MSE(y_test, y_pred_test)  
print(f'MSE: {mse_value}')

In [ ]:
def SA_base_on_MSE(model, x_Model, y_Model):
    ### stepwise selection II is to set all values of one input variable to its mean (or others) 
    ### and examine the value of MSE under the change.

    SA = pd.Series(index = x_Model.columns, dtype='object'  ) # create a table
    x_train, x_test,x_val, y_train, y_test,y_val, x_scaled, y_scaled, ind_train, ind_test,ind_val= split(xx, yy)
    # replace one parameter's values into its mean and examine its efficiency

    for i in range(0, x_scaled.shape[1]) :
        x_test_r = np.array(x_scaled) # copy x_test
        x_test_r[:,i] = x_scaled[:,i].mean()
        y_re_test=model.predict(x_test_r)
        SA[i] = MSE(y_scaled, y_re_test)
    return SA


In [ ]:
SA_GA_pred = SA_base_on_MSE(ga_svm,xx,yy)
SA_GPR_pred = SA_base_on_MSE(gpr,xx,yy)
SA_XGB_pred = SA_base_on_MSE(xgb,xx,yy)
SA_RF_pred = SA_base_on_MSE(rf,xx,yy)

x = np.arange(len(SA_GA_pred.index))  # the label locations     #SA_RF_pred替换SA_GA_pred
width = 0.15  # the width of the bars

fig, ax = plt.subplots(figsize=(20, 6))
rects1 = ax.bar(x - 2*width, SA_GA_pred.values, width, label='GA-SVM')  
rects2 = ax.bar(x - width, SA_XGB_pred.values, width, label='XGB')  
rects3 = ax.bar(x + width, SA_GPR_pred.values, width, label='GPR')  
rects4 = ax.bar(x, SA_RF_pred.values, width, label='RF')  

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('RMSE')
ax.set_title('Sensitive Analysis')
ax.set_xticks(x)
ax.tick_params(axis='x', rotation=0)
ax.set_xticklabels(SA_RF_pred.index)  # 假设使用 RF 的索引作为 x 轴标签  
plt.grid(False)     # 禁用网格  
ax.legend(loc='best')

In [ ]:
SA_GA_pred.values

# Sobol index with Scipy generate samples

bounds = pd.DataFrame(index=xx.columns,columns=['min','max'])
bounds['min'] = np.min(xx,0)
bounds['max'] = np.max(xx,0)
bounds

from SALib.sample import saltelli
from SALib.analyze import sobol
from SALib.test_functions import Ishigami
import numpy as np

# Define the problem
problem = {
    'num_vars': len(xx.columns),
    'names': ['Time', 'Deep', 'Length', 'HRT', 'COD', 'TN', 'NH4-N', 'C/N', 'c(A)','CSC','XLogP3','WI','RA(P)','RA(F)','RA(B)','RA(A)'],
    'bounds': bounds.values.astype('float')
}
param_values = saltelli.sample(problem,1024)
param_values.shape

Y1 = rf.predict(param_values)
Si1 = sobol.analyze(problem,Y1,print_to_console=True)
total_si1,first_si1,second_si1 = Si1.to_df()

# 从Si字典中提取总敏感性贡献度（S1）和其他你需要的指标  
S1 = Si1['ST']  # 根据SALib的版本和具体实现，这里可能需要调整 

# 如果需要，将S1转换为DataFrame  
df_sensitivity = pd.DataFrame(S1, index=problem['names'], columns=['ST']) 
  

print(type(df_sensitivity))

# 使用正确的列名，通常总敏感性贡献度列名为 'S1'  
total_si_column = 'ST'  # 总敏感性贡献度的列名  
  
# 提取总敏感性贡献度和对应的特征名称  
total_si1 = df_sensitivity[total_si_column]  
features = df_sensitivity.index.tolist()  # 假设特征名称在索引中  

# 确保总敏感性贡献度之和为1，如果不是，则进行归一化处理  
total_si1_normalized = total_si1 / total_si1.sum()  
  
# 绘制饼图  
fig, ax = plt.subplots()  
ax.pie(total_si1_normalized, labels=features, autopct='%1.1f%%', startangle=90)  
  
# Equal aspect ratio ensures that pie is drawn as a circle.  
ax.axis('equal')    
  
# 设置标题  
plt.title('Feature Contribution to Output Variability')  
  
# 显示饼图  
plt.show()

Y2 = xgb.predict(param_values)
Si2 = sobol.analyze(problem,Y2,print_to_console=True)
total_si2,first_si2,second_si2 = Si2.to_df()

# 提取总敏感性贡献度  
S1 = total_si2['ST']  
  
# 创建DataFrame  
df_sensitivity = pd.DataFrame(S1, index=problem['names'], columns=['ST'])  
  
# 归一化总敏感性贡献度  
df_sensitivity_normalized = df_sensitivity / df_sensitivity['ST'].sum()  
  
# 绘制饼图  
fig, ax = plt.subplots()  
ax.pie(df_sensitivity_normalized['ST'], labels=df_sensitivity.index.tolist(), autopct='%1.1f%%', startangle=90)  
ax.axis('equal')  # 确保饼图是圆形  
plt.title('Feature Contribution to Output Variability')  
plt.show()

Y3 = ga_svm.predict(param_values)
Si3 = sobol.analyze(problem,Y3,print_to_console=True)
total_si3,first_si3,second_si3 = Si3.to_df()

# 从Si字典中提取总敏感性贡献度（S1）和其他你需要的指标  
S1 = Si3['ST']  # 根据SALib的版本和具体实现，这里可能需要调整 

# 如果需要，将S1转换为DataFrame  
df_sensitivity = pd.DataFrame(S1, index=problem['names'], columns=['ST']) 

Y4 = gpr.predict(param_values)
Si4 = sobol.analyze(problem,Y4,print_to_console=True)
total_si4,first_si4,second_si4 = Si4.to_df()

from SALib.sample import saltelli
from SALib.analyze import sobol
from SALib.test_functions import Ishigami
import numpy as np

# Define the problem
problem = {
    'num_vars': len(xx.columns),
    'names': ['Time', 'Deep', 'Length', 'HRT', 'COD', 'TN', 'NH4-N', 'C/N', 'c(A)','CSC','XLogP3','WI','RA(P)','RA(F)','RA(B)','RA(A)'],
    'bounds': bounds.values.astype('float')
}

param_values = saltelli.sample(problem,1024)

param_values
param_values.shape
Y_gpr2 = gpr.predict(param_values) 
Y_svm2 = ga_svm.predict(param_values) 
Y_rf2 = rf.predict(param_values)
Si_gpr2 = sobol.analyze(problem, Y_gpr2, print_to_console=True)
Si_svm2 = sobol.analyze(problem, Y_svm2, print_to_console=True)
Si_rf2 = sobol.analyze(problem, Y_rf2, print_to_console=True)
total_Si_gpr, first_Si_gpr, second_Si_gpr = Si_gpr2.to_df()
total_Si_svm, first_Si_svm, second_Si_svm = Si_svm2.to_df()
total_Si_rf, first_Si_rf, second_Si_rf = Si_rf2.to_df()


x = np.arange(len(total_si1.index))  # the label locations   #total_Si_rf替换total_Si_svm
width = 0.2  # the width of the bars

fig, ax = plt.subplots(figsize=(15, 6))
rects1 = ax.bar(x-width, abs(first_si3['S1']), width, label='GA-SVM')
rects2 = ax.bar(x+width, abs(first_si2['S1']), width, label='XGB')
#rects3 = ax.bar(x+width, abs(first_si2['S1']), width, label='GPR')
rects4 = ax.bar(x, abs(total_si1['S1']), width, label='RF')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Sobol index')
ax.set_title('first sobol values')
ax.set_xticks(x)
ax.set_xticklabels(total_si1.index)  #total_Si_rf替换total_Si_svm
ax.tick_params(axis='x', rotation=0)  # 这里将旋转角度设置为45度
ax.legend(loc='best')

x = np.arange(len(total_si3.index))  # the label locations
width = 0.2  # the width of the bars

fig, ax = plt.subplots(figsize=(15, 6))
rects1 = ax.bar(x-width, total_si3['ST'], width, label='GA-SVM')
rects2 = ax.bar(x, total_si1['ST'], width, label='RF')
rects3 = ax.bar(x+width, total_si2['ST'], width, label='XGB')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Sobol index')
ax.set_title('total sobol values')
ax.set_xticks(x)
ax.set_xticklabels(total_si3.index)
ax.tick_params(axis='x', rotation=0)  # 这里将旋转角度设置为45度
ax.legend(loc='best')

df_sobol = pd.DataFrame(index=SA_GA_pred.index, columns = ['GPR','GA-SVM','RF'])
df_sobol['GPR'] = total_si2['ST'].values
df_sobol['GA-SVM'] = total_si3['ST'].values
df_sobol['RF'] = total_si1['ST'].values



# SHAP (SHapley Additive exPlanations)

In [66]:
df_shap = pd.DataFrame(index=xx.columns, columns = ['ga_svm','rf','xgb','gpr'])

In [67]:
import shap
np.random.seed(42)
#ga_svm
explainer = shap.KernelExplainer(ga_svm.predict, x_test)    
shap_values_ga_svm = explainer.shap_values(x_test)      
df_shap['ga_svm'] = np.average(abs(shap_values_ga_svm),0)   
shap.summary_plot(shap_values_ga_svm,  xx, plot_type="bar") 

NameError: name 'ga_svm' is not defined

In [ ]:
# 假设你已经有了每个特征的平均SHAP绝对值  
feature_importances = df_shap['ga_svm']  
  
# 归一化特征重要性，使其和为1  
feature_importances_norm = feature_importances / feature_importances.sum()  
  
# 获取特征名称  
feature_names = df_shap.index  
 
# 需要描边的扇形索引列表  
wedges_to_highlight = [5,6,7,15]  # 假设你想要突出显示第2、4、6个扇形（索引从0开始）  
# 绘制饼状图  
fig, ax = plt.subplots()  
explode = ( 0.02, 0.02, 0.02, 0.02, 0.02, 0.05, 0.05, 0.05, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.05)
wedges, texts, autotexts = ax.pie(feature_importances_norm, labels=feature_names, autopct='%1.1f%%', pctdistance=0.85, labeldistance=1.1, startangle=-60, radius=0.5, explode=explode, wedgeprops=dict(edgecolor='w', linewidth=1))  # 初始设置所有扇形的描边  
# 为特定的扇形设置不同的描边效果  
for i in wedges_to_highlight:  
    wedges[i].set_edgecolor('black')  # 设置描边颜色为红色  
    wedges[i].set_linewidth(2)  # 设置描边宽度为2  

ax.axis('equal')  # 确保饼图是圆形的  
ax.set_title('Feature Importance-ga_svm', pad=25)  # 设置标题  
#plt.savefig('TN_Feature Importance-GA_SVM.png', dpi=500, transparent=True)  # 保存图片到当前目录，dpi参数设置图片分辨率
plt.show()

In [ ]:
#gpr
np.random.seed(42)
explainer = shap.KernelExplainer(gpr.predict,  x_test)      
shap_values_gpr = explainer.shap_values(x_test)             
# visualize the first prediction's explanation
df_shap['gpr'] = np.average(abs(shap_values_gpr),0)          
shap.summary_plot(shap_values_gpr,  xx, plot_type="bar")    

In [ ]:
# 假设你已经有了每个特征的平均SHAP绝对值  
feature_importances = df_shap['gpr']  
# 归一化特征重要性，使其和为1  
feature_importances_norm = feature_importances / feature_importances.sum()  
# 获取特征名称  
feature_names = df_shap.index  

# 需要描边的扇形索引列表  
wedges_to_highlight = [5,6,7,13,15]  # 假设你想要突出显示第2、4、6个扇形（索引从0开始）  
# 绘制饼状图  
fig, ax = plt.subplots()  
explode = ( 0.02, 0.02, 0.02, 0.02, 0.02, 0.05, 0.05, 0.05, 0.02, 0.02, 0.02, 0.02, 0.02, 0.05, 0.02, 0.05)
wedges, texts, autotexts = ax.pie(feature_importances_norm, labels=feature_names, autopct='%1.1f%%', pctdistance=0.85, labeldistance=1.1, startangle=-45, radius=1, explode=explode)  # 初始设置所有扇形的描边  
  
# 为特定的扇形设置不同的描边效果  
for i in wedges_to_highlight:  
    wedges[i].set_edgecolor('black')  # 设置描边颜色为红色  
    wedges[i].set_linewidth(2)  # 设置描边宽度为2  

ax.axis('equal')  # 确保饼图是圆形的  
ax.set_title('Feature Importance-GPR', pad=25)  # 设置标题  
#plt.savefig('TN-Feature Importance-GPR.png', dpi=500, transparent=True)  # 保存图片到当前目录，dpi参数设置图片分辨率
plt.show()

In [68]:
#xgb
import shap
np.random.seed(42)
explainer = shap.KernelExplainer(xgb.predict,  x_test)      #rf替换gpr
shap_values_xgb = explainer.shap_values(x_test)             #rf替换gpr
# visualize the first prediction's explanation
#df_shap['xgb'] = np.average(abs(shap_values_xgb),0)          #rf替换gpr
#shap.summary_plot(shap_values_xgb,  xx, plot_type="bar")    #rf替换gpr

feature_names = ['Time', 'Deep', 'Length', 'HRT', 'COD', 'TN', 'NH4-N', 'C/N', 'c(A)', 'CSC', 'XLogP3', 'WI', 'RA(P)', 'RA(F)', 'RA(B)', 'RA(A)']  
shap.summary_plot(shap_values_xgb, x_test, feature_names=feature_names)
ax.set_xlim(-0.30, 0.30)  

plt.show()

  0%|          | 0/50 [00:00<?, ?it/s]

D:\Python\Lib\site-packages\shap\plots\_beeswarm.py:952: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  pl.show()
C:\Users\FFF123\AppData\Local\Temp\ipykernel_9264\3664686764.py:13: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [69]:
# 保存为SVG格式  
plt.savefig('111XGB-TN-shap_summary_plot.svg',format='svg')

In [ ]:
# 假设你已经有了每个特征的平均SHAP绝对值  
feature_importances = df_shap['xgb']  
# 归一化特征重要性，使其和为1  
feature_importances_norm = feature_importances / feature_importances.sum()  
# 获取特征名称  
feature_names = df_shap.index  

# 需要描边的扇形索引列表  
wedges_to_highlight = [1,4,7,12]  # 假设你想要突出显示第2、4、6个扇形（索引从0开始）  
# 绘制饼状图  
fig, ax = plt.subplots()  
explode = ( 0.02, 0.05, 0.02, 0.02, 0.05, 0.02, 0.02, 0.05, 0.02, 0.02, 0.02, 0.02, 0.05, 0.02, 0.02, 0.02)
wedges, texts, autotexts = ax.pie(feature_importances_norm, labels=feature_names, autopct='%1.1f%%', pctdistance=0.85, labeldistance=1.1, startangle=90, radius=0.5, explode=explode)  # 初始设置所有扇形的描边  
  
# 为特定的扇形设置不同的描边效果  
for i in wedges_to_highlight:  
    wedges[i].set_edgecolor('black')  # 设置描边颜色为红色  
    wedges[i].set_linewidth(2)  # 设置描边宽度为2  

ax.axis('equal')  # 确保饼图是圆形的 
ax.set_title('Feature Importance-XGB', pad=25)  # 设置标题  
#plt.savefig('TN-Feature Importance-RF.png', dpi=500, transparent=True)  # 保存图片到当前目录，dpi参数设置图片分辨率
plt.show()

In [ ]:
#rf
np.random.seed(42)
explainer = shap.KernelExplainer(rf.predict,  x_test)      #rf替换gpr
shap_values_rf = explainer.shap_values(x_test)             #rf替换gpr
# visualize the first prediction's explanation
#df_shap['rf'] = np.average(abs(shap_values_rf),0)          #rf替换gpr
#shap.summary_plot(shap_values_rf,  xx,plot_type="bar")    #rf替换gpr

In [ ]:
import shap
np.random.seed(42)
explainer = shap.KernelExplainer(rf.predict,  x_test)      #rf替换gpr
shap_values_rf = explainer.shap_values(x_test)    
feature_names = ['Time', 'Deep', 'Length', 'HRT', 'COD', 'TN', 'NH4-N', 'C/N', 'c(A)', 'CSC', 'XLogP3', 'WI', 'RA(P)', 'RA(F)', 'RA(B)', 'RA(A)']  
shap.summary_plot(shap_values_rf, x_test, feature_names=feature_names)
ax.set_xlim(-0.30, 0.30)  

In [ ]:
plt.show()

In [ ]:
# 保存为SVG格式  
plt.savefig('RF-CA-shap_summary_plot.svg',format='svg')

In [ ]:
# 假设你已经有了每个特征的平均SHAP绝对值  
feature_importances = df_shap['rf']  
# 归一化特征重要性，使其和为1  
feature_importances_norm = feature_importances / feature_importances.sum()  
# 获取特征名称  
feature_names = df_shap.index  

# 需要描边的扇形索引列表  
wedges_to_highlight = [1,4,5,7,12]  # 假设你想要突出显示第2、4、6个扇形（索引从0开始）  
# 绘制饼状图  
fig, ax = plt.subplots()  
explode = ( 0.02, 0.05, 0.02, 0.02, 0.05, 0.05, 0.02, 0.05, 0.02, 0.02, 0.02, 0.02, 0.05, 0.02, 0.02, 0.02)
wedges, texts, autotexts = ax.pie(feature_importances_norm, labels=feature_names, autopct='%1.1f%%', pctdistance=0.85, labeldistance=1.1, startangle=90, radius=0.5, explode=explode)  # 初始设置所有扇形的描边  
  
# 为特定的扇形设置不同的描边效果  
for i in wedges_to_highlight:  
    wedges[i].set_edgecolor('black')  # 设置描边颜色为红色  
    wedges[i].set_linewidth(2)  # 设置描边宽度为2  

ax.axis('equal')  # 确保饼图是圆形的 
ax.set_title('Feature Importance-RF', pad=25)  # 设置标题  
#plt.savefig('TN-Feature Importance-RF.png', dpi=500, transparent=True)  # 保存图片到当前目录，dpi参数设置图片分辨率
plt.show()

In [ ]:
# explain the model's predictions using SHAP
# (same syntax works for LightGBM, CatBoost, scikit-learn, transformers, Spark, etc.)
explainer = shap.Explainer(xgb)    #rf替换xgbr
shap_values_xgb = explainer(x_test)    #rf替换xgbr
df_shap['xgb'] = np.average(abs(shap_values_xgb.values),0)    #rf替换xgbr
# visualize the first prediction's explanation
shap.plots.bar(shap_values_xgb)    #rf替换xgbr

In [ ]:
x = np.arange(len(df_shap.index))  # the label locations
width = 0.15  # the width of the bars

fig, ax = plt.subplots(figsize=(15, 6))
rects1 = ax.bar(x - 2*width, df_shap['ga_svm'], width, label='GA-SVM')
rects2 = ax.bar(x - width, df_shap['xgb'], width, label='XGB')
rects3 = ax.bar(x+width, df_shap['gpr'], width, label='GPR')
rects4 = ax.bar(x, df_shap['rf'], width, label='RF')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('shap value')
ax.set_title('SHapley Additive exPlanations')
ax.set_xticks(x)
ax.set_xticklabels(df_shap.index)
ax.tick_params(axis='x', rotation=0)  # 这里将旋转角度设置为45度
ax.legend(loc='best')

In [ ]:
df_shap

#   偏相关依赖图PDP

In [ ]:
from matplotlib import pyplot as plt
from sklearn.inspection import PartialDependenceDisplay

x_train, x_test_val, y_train, y_test_val = train_test_split(xx, yy, shuffle=True,test_size=0.3, random_state=1)
    #x_训练集，x_测试验证集，y_训练集，y_测试验证集                    划分前将数据集进行随机打乱，并按照7：3划分，且划分方式统一
x_test, x_val, y_test, y_val = train_test_split(x_test_val, y_test_val, shuffle=True,test_size=0.25, random_state=1)
    #x_测试集，x_验证集，y_测试集，y_验证集                           划分前将数据集进行随机打乱，并按照3：1划分，且划分方式统一
    
feature_names = ['Time', 'Deep', 'Length', 'HRT', 'COD', 'TN', 'NH4-N', 'C/N', 'c(A)', 'CSC', 'XLogP3', 'WI', 'RA(P)', 'RA(F)', 'RA(B)', 'RA(A)']  

feat_name = ('Time')
PartialDependenceDisplay.from_estimator(rf, x_train, [feat_name])
plt.gcf().set_size_inches(6, 6)  # w 是宽度，h 是高度  
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
f_names = [('XLogP3', 'RA(A)')]
# Similar to previous PDP plot except we use tuple of features instead of single feature
disp4 = PartialDependenceDisplay.from_estimator(rf,x_test, f_names, ax=ax)
plt.show()
